<a href="https://colab.research.google.com/github/gtbook/robotics/blob/main/S75_drone_decision_theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q -U gtbook


In [ ]:
import numpy as np
import plotly.express as px
import gtsam
from gtsam import Point2

from gtbook import diffdrive
from gtbook.discrete import Variables
from gtbook.display import show, pretty

import torch
DEVICE = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

rng = np.random.default_rng()


In [ ]:
def create_random_map(W, H, num_obstacles=50, seed=42):
    """
    Creates a random occupancy map with the given dimensions and number of obstacles.
    Optionally, a seed can be provided to make the map reproducible.
    """
    # Set the seed for reproducibility:
    rng = np.random.default_rng(seed=seed)

    # Map dimensions in pixels (10 cm per pixel)
    pixel_W, pixel_H = 10*W, 10*H

    # Initialize an empty occupancy map
    obstacles = torch.zeros((pixel_H, pixel_W), dtype=torch.float32)

    # Add obstacles
    for _ in range(num_obstacles):
        # Random position for the obstacle
        x, y = rng.integers(0, pixel_W), rng.integers(0, pixel_H)

        # Random value for the obstacle (higher than empty space)
        obstacle_value = rng.random() * 0.5 + 0.5  # Random value between 0.5 and 1.0

        # Assign the value to the obstacle position
        obstacles[y, x] = obstacle_value

    # Define a box filter (kernel)
    kernel_size = 5  # The size of the box filter
    kernel = torch.ones((kernel_size, kernel_size), dtype=torch.float32)

    # Apply 2D convolution to "fatten" the objects
    batch = obstacles[None, None, ...]  # Add batch and channel dimensions
    return F.conv2d(batch, kernel[None, None, ...], padding='same')[0, 0, ...]

In [ ]:
def displaced_gaussian(
    sigma: float, K: int, uv: np.ndarray
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Returns a Gaussian kernel of size K x K with standard deviation sigma.
    The kernel is centered at uv, a float coordinate in xy convention.
    Also returns the indices of the kernel in the image.
    """
    # Create meshgrid around uv, in ij convention
    j0, i0 = np.round(uv).astype(int)
    kk = np.arange(-(K // 2), K // 2 + 1)
    ii, jj = np.meshgrid(i0 + kk, j0 + kk, indexing="ij")

    # Calculate the kernel
    dd = (ii - uv[1]) ** 2 + (jj - uv[0]) ** 2
    kernel = np.exp(-dd / (2 * sigma**2))
    kernel = kernel / kernel.sum()

    # Return the kernel and the indices of the kernel in the image
    return kernel, ii, jj


def gaussian_kernel(sigma: float, K: int = 9) -> torch.Tensor:
    """
    Generates a Gaussian kernel of size K x K with standard deviation sigma.
    """
    uv = np.array([K // 2, K // 2])
    numpy_kernel, _, _ = displaced_gaussian(sigma, K, uv)
    return torch.tensor(numpy_kernel, dtype=torch.float32)[None, None, ...]


def gaussian_filter(sigma, uv, image, K=9):
    """
    Applies a Gaussian filter at uv on the given image.
    """
    kernel, ii, jj = displaced_gaussian(sigma, K, uv)
    return (kernel * image[..., ii, jj].numpy()).sum(axis=(-2, -1))

# Trajectory Optimization

> We can use factor graphs to optimize over future trajectories, as well.

<img src="Figures7/S75-Autonomous_camera_drone-05.jpg"  alt="Splash image with steampunk drone in aggressive maneuver" width="40%" align=center style="vertical-align:middle;margin:10px 0px">


In the previous section we saw how do use factor graphs for visual SLAM and structure from motion. These perception algorithms are typically run after we have gathered some visual information, and inform us about what happened in the past. But how can we plan for the future? We already saw that RRTs are useful paradigm to plan in a continuous, potentially high dimensional state space. 

In this section we will discuss a different method, which is optimal control. In particular, we will use **trajectory optimization**, to minimize the cost associated with the trajectory that we would like to execute in the future. These costs can be associated with staying away from obstacles, or other desirable properties like minimizing power consumption to preserve battery life. Finally, RRTs and trajectory optimization can be combined, where RRT finds a good "broad strokes" trajectory, and trajectory optimization smooths out and fine-tunes the final trajectory.

## A Simple Objective

We explain trajectory optimization using a very simple problem: how to get from point A to point B. This seems like an easy problem, as we all know the answer should be a straight line, right? But, it is not as easy for drones, as we have to account for the drone's attitude, and we might have to accelerate and decelerate at the start and end points. Come to think of it, maybe we are already flying at a certain velocity at point A, and want to have a certain velocity at point B! Clearly, this is not as easy as it appears.

There are many ways to go about this:
- MPC with collocated polynomials
- discrete poses and controls + dynamics factors
- differential flatness + controller: this is what we'll do

## Optimizing for Position

It turns out we can optimize for position only, without worrying about attitude, at least at first. This is not a trivial statement, and it is also very fortunate. Let us assume that we want to fly in a forest, full of trees, and we want a trajectory for the drone avoid the trees. A quadrotor-style drone trajectory has the potential to be rather complicated: the drone lives in the 6DOF space of position (3DOF) and attitude (3DOF). And we know from Section 7.2 that attitude will *affect* position: if we pitch or role, that will affect our velocity, and that will integrate into position. That very mechanism is also a source of complexity: a drone is *under-actuated*: we cannot directly control the 6DOF, we have to do it through the action only four rotors. It seems like a very complex problem indeed.

When given a continuous trajectory of position that also behaves well in terms of velocity, we can obtain the pitch, roll, and thrust trajectories to obtain it. Of course, it can't be a totally crazy trajectory: a drone can't stop on a dime, or teleport, or go faster than certain limits. But if we have a 3DOF translational trajectory that is well behaved, we can "upgrade" it to a full position/attitude in two different ways: in advance, by solving a large optimization problem, or physically, in real-time, by using a tracking controller. The latter is what we will do, after obtaining a trajectory.

To find a suitable translational trajectory that avoids obstacles, we will use optimization. There are many ways one can do this, some better than others, but here strive for simplicity: we discretize the time interval in which we expect to fly and solve for a set of corresponding set of positions over time that satisfy our objectives. These objectives can include:

- a desired starting position (and velocity, if so desired);
- a desired goal position (and velocity, if desired);
- a desired distance from the closest obstacle at any time;
- a smoothness objective that bounds our velocity and/or acceleration.

If these objectives are in conflict, we can assign them weights such that we trade off some against others. Constrained optimization techniques allow to set hard constraints and/or bounds, but we do not use those here. Instead, we opt for a simple non-linear least squares scheme that minimizes the following objective:

$$
X^* = \arg \min \sum_{k=1}^K \phi_k(X_k)^2  + \sum_{k=1}^{K-1} \psi_k(X_k, X_{k+1})^2
$$

where $X_k$ is the position at time, and $\phi_k(X_k)$ and $\psi_k(X_k, X_{k+1})$ are unary and binary objectives, respectively, that we want to minimize. Desired start and goal, as well as obstacle avoidance are examples of unary objectives, while smoothness is a binary objective, as it evaluates the distance/velocity between successive positions.

It will not come as any surprise that we can also use factor graph optimization to find our optimal trajectories. The objective above is exactly the expression of a nonlinear factor graph, except that the factors now derive from objectives we want to minimize, rather than measurement errors. In the next section we work in detail how to do so.

## Factor Graphs for Trajectory Optimization

A very popular approach to represent environments with different obstacles is to use an **occupancy map**. An occupancy map is essentially a 2D array of cells, each representing the environment at a certain resolution (for example, 10 cm by 10 cm). Each cell in this map contains a probability for the presence of an obstacle. This concept is further extendable to distinguish between various types of obstacles, each with different levels of lethality. This extended version can then be transformed into a **cost map**, where areas with less hazardous obstacles (like tall grass) are considered less costly compared to more dangerous ones (like flying into a tree). For our purposes, we will start directly with cost maps instead of creating true occupancy maps.

To illustrate, let us consider creating a simple example cost map. In the `gtbook` library, we have a function named `create_random_map`. This function requires as parameters the width $W$, height $H$, and the number of obstacles to create a random arrangement of obstacles within the map. Width and height are given in meters, and the resolution is hardcoded at 10 cm. An optional parameter is a random seed, which allows for the reproduction of the exact experiment setup. The following code snippet demonstrates how to generate a random map with 50 obstacles using a random seed of 42:

In [ ]:
# Create a random map
W, H = 30, 10  # 30m x 10m
occupancy_map = create_random_map(W, H, num_obstacles=50, seed=42)
px.imshow(occupancy_map, color_continuous_scale='Reds').show()

To create differentiable factors $\phi_k(X_k)$ that avoid high cost areas, we can smooth the cost map with a differentiable kernel, like a Gaussian. The cost map above is a torch tensor, representing a grayscale image 100 pixels tall and 300 pixels wide. Convolving an image with a kernel is relatively easy in pytorch, though we need to juggle some tensor dimensions because by convention, `conv2d` operates on $\text{batch} \times \text{channel} \times \text{height} \times \text{width}$ tensors:

In [ ]:
kernel = gaussian_kernel(sigma=0.5*10, K=21)
convolved = torch.conv2d(occupancy_map[None, None, ...], kernel, padding='same')[0, 0, ...]
px.imshow(convolved, color_continuous_scale='Reds').show()

In [ ]:
# Compute gradients:
sobel_u = torch.tensor([[-1, 0, 1]], dtype=float)
sobel_v = sobel_u.T
grad_u = torch.conv2d(occupancy_map, sobel_u[None, None, ...], padding='same')
grad_v = torch.conv2d(occupancy_map, sobel_v[None, None, ...], padding='same')
combined = torch.stack([occupancy_map, grad_u, grad_v], dim=-1)[0, 0 ,...]
px.imshow(combined[...,1], zmin=-1, zmax=1)

In [ ]:
batch = combined.permute(2, 0, 1)[None, ...]
blurred = torch.conv2d(batch, kernel_tensor, padding="same", groups=3)
px.imshow(blurred[0,0,...], zmin=-1, zmax=1, color_continuous_scale='rdbu')

In [ ]:
# Assert same result from blurred image
uv = Point2(59, 11)
displaced_kernel, ii, jj = displaced_gaussian(sigma, K, uv)
displaced = (displaced_kernel * batch[..., ii, jj].numpy()).sum(axis=(-2, -1))

# assert kernels are the same:
assert np.allclose(displaced_kernel, kernel)

# assert blurred image gives the same result:
assert np.allclose(displaced, blurred[..., int(uv[1]), int(uv[0])])

In [ ]:
# Generate a random 2D position within the map as floats
position = rng.random(2) * [W, H]
position

In [ ]:
gaussian_filter(sigma=10*0.2, K=9, uv=position*10, image=batch)

## Smoothing

- A magic Chebyshev interpolation class that gets us derivatives and accelerations
- Can also be used for RRT, see project

## Translational Control

- A thrust controller
- A pitch/roll controller
  - calculate desired acceleration
  - from that we calculate thrust vector we need, compensate for gravity
  - project onto the F and L body axes

## Planning and Controlling Yaw

- We'd like to point in a particular direction
- forward looking, or camera drone
- Explain why controlling yaw is harder
- A simple yaw controller